In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [6]:

# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.001
FINE_TUNE_EPOCHS = 20  # Tambahan epoch untuk fine-tuning
FINE_TUNE_LR = 1e-5    # Learning rate untuk fine-tuning

# Direktori dataset
train_dir = "train_batik"
val_dir = "val_batik"
test_dir = "test_batik"

# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

val_datagen = ImageDataGenerator(rescale=1.0/255)


# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 928 images belonging to 29 classes.


Found 116 images belonging to 29 classes.
Found 116 images belonging to 29 classes.


In [7]:


# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar untuk transfer learning

# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model untuk transfer learning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


In [ ]:

# Pelatihan awal (transfer learning)
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)



c:\Users\MAHASISWA\Documents\GitHub\Nusantara_Culture_Database\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.0628 - loss: 3.6726 - val_accuracy: 0.2931 - val_loss: 2.6625 - learning_rate: 0.0010
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.3267 - loss: 2.4638 - val_accuracy: 0.3276 - val_loss: 2.2675 - learning_rate: 0.0010
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.4403 - loss: 1.9229 - val_accuracy: 0.4483 - val_loss: 1.8668 - learning_rate: 0.0010
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.5649 - loss: 1.5630 - val_accuracy: 0.4483 - val_loss: 1.7438 - learning_rate: 0.0010
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 999ms/step - accuracy: 0.5792 - loss: 1.4201 - val_accuracy: 0.5086 - val_loss: 1.7756 - learning_rate: 0.0010
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6403 - loss: 1.2370 - val_accuracy: 0.5345 - val_loss: 1.6878 - learning_rate: 0.0010
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6790 - loss: 1.1204 - val_accurac

In [9]:
# Fine-tuning
# Buka beberapa lapisan terakhir dari base model
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan semua kecuali 20 lapisan terakhir
    layer.trainable = False

# Kompilasi ulang model dengan learning rate yang lebih kecil
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning model
history_fine_tune = model.fit(
    train_generator,
    epochs=FINE_TUNE_EPOCHS,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.6033 - loss: 1.3311 - val_accuracy: 0.6121 - val_loss: 1.4836 - learning_rate: 1.0000e-05
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.6514 - loss: 1.2177 - val_accuracy: 0.6121 - val_loss: 1.4829 - learning_rate: 1.0000e-05
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.6480 - loss: 1.1556 - val_accuracy: 0.6121 - val_loss: 1.4865 - learning_rate: 1.0000e-05
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6758 - loss: 1.1240 - val_accuracy: 0.6293 - val_loss: 1.4918 - learning_rate: 1.0000e-05
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.6365 - loss: 1.1644 - val_accuracy: 0.6207 - val_loss: 1.4962 - learning_rate: 1.0000e-05
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 673ms/step - accuracy: 0.6527 - loss: 1.4259
Test Loss: 1.3717325925827026
Test Accuracy: 0.6551724076271057


In [7]:
model.save("model_10_20_epochs.h5")  # Menyimpan model ke file my_model.h5